In [1]:
import nltk, pandas as pd, numpy as np
from nltk.parse.corenlp import CoreNLPParser, CoreNLPDependencyParser
from nltk.tree import ParentedTree
from SPARQLWrapper import SPARQLWrapper, JSON
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')
pos_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='pos')

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
def runQuery(query,res):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query) 
    sparql.setReturnFormat(JSON)
    result = sparql.query().convert()
    for hit in result["results"]["bindings"]:
        res[hit["x1"]["value"]]=True;
    for i in list(res.keys()):
        print(i)
def sparqlQuery(a,b):
    res=dict()
    s1=""" PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX quepy: <http://www.machinalis.com/quepy#>
    PREFIX dbpedia: <http://dbpedia.org/ontology/>
    PREFIX dbpprop: <http://dbpedia.org/property/>
    PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>"""
    s2="""
    SELECT DISTINCT ?x1 WHERE {
    ?x0 rdfs:label """
    s3="\""+a+"\"" +"@en."
    s4=""
    s5=""
    if(b !=""):
        s4="""?x0 dbo:"""+b+""" ?x1.}LIMIT 1"""
        s5="""?x0 dbp:"""+b+""" ?x1.}LIMIT 1"""
        runQuery(s1+s2+s3+s4,res)
        runQuery(s1+s2+s3+s5,res)
    else:
        s2="""
        SELECT DISTINCT ?x1 WHERE {
        ?x1 rdfs:label """
        s3="\""+a+"\"" +"@en.}LIMIT 1"
        runQuery(s1+s2+s3,res)
sparqlQuery("India","flower")

http://dbpedia.org/resource/Nelumbo_nucifera


In [3]:
def triplet_extraction (input_sent, output=['parse_tree','spo','result']):
    # Parse the input sentence with Stanford CoreNLP Parser
    pos_type = pos_tagger.tag(input_sent.split())
    parse_tree, = ParentedTree.convert(list(pos_tagger.parse(input_sent.split()))[0])
    dep_type, = ParentedTree.convert(dep_parser.parse(input_sent.split()))
    # Extract subject, predicate and object
    x,y,z= extract_subject(parse_tree)
    predicate = extract_predicate(parse_tree)
 
    return x,y,z,predicate;

def extract_subject (parse_tree):
    # Extract the first noun found in NP_subtree
    subject = []
  #  print(parse_tree)
    f=False;
    ff=False
    temp=""
    x=''
    y=''
    z=[]
    for s in parse_tree.subtrees(lambda x: x.label() == 'NP'):
        if(not f):
            temp=' '.join(s.flatten())
            f=True
            x=temp
        for t in s.subtrees(lambda y: y.label().startswith('NN')):
            if(not ff):
                y=t[0];
                z=extract_attr(t)
                ff=True
    return x,y,z
    

def extract_predicate (parse_tree):
    # Extract the deepest(last) verb foybd ub VP_subtree
    output, predicate = [],[]
    for s in parse_tree.subtrees(lambda x: x.label() == 'VP'):
        for t in s.subtrees(lambda y: y.label().startswith('VBD') or y.label().startswith('VBZ')):
            output = [t[0], extract_attr(t)]
            if output != [] and output not in predicate:    
                predicate.append(output)
    if len(predicate) != 0: return predicate[-1]
    else: return
def extract_attr (word):
    attrs = []     
  #  print(word.parent())
    # Search among the word's siblings
    if word.label().startswith('JJ'):
        for p in word.parent(): 
            if p.label() == 'RB':
                attrs.append(p[0])
    elif word.label().startswith('VB'):
        for p in word.parent():
            if p.label() == 'ADVP':
                attrs.append(p[0])
    # Search among the word's uncles
    if word.label().startswith('NN'):
        for p in word.parent().parent():
            if p.label() == 'PP':
                for t in p.subtrees(lambda y: y.label()=='NP'):
                     attrs.append(' '.join(t.flatten()))
       
    return attrs
triplet_extraction('Who is the host of BBC Wild Life Specials',['parse_tree','spo','result'])
triplet_extraction('Who is Rabindranath Tagore')
triplet_extraction('who wrote the pillier of earth')
triplet_extraction('who directs pather panchali')
triplet_extraction('who is the author of the origin of the species')
triplet_extraction('what is the total population of Melbourne')
triplet_extraction('what are the launch pad operated in NASA')
triplet_extraction('which companies are located in California')
triplet_extraction('what is the location of House of Parliament')
triplet_extraction('what is the age of Rabindranath Tagore')
triplet_extraction('who have pen')



('pen', 'pen', [], None)

In [4]:
def findAnswer(question):  
    x,y,z,predicate=triplet_extraction(question)

    print("result")
    if(predicate==None):
        lable=x
        relation=""
        sparqlQuery(lable,relation)
        #calltriplet_extraction('what is the age of Rabindranath Tagore')
        for i in z :
            lable=i
            relation=y
            sparqlQuery(lable,relation)
    else:
        lable=x
        relation=predicate[0]
        sparqlQuery(lable,relation)
        for i in z :
            lable=i
            relation=predicate[0]
            sparqlQuery(lable,relation)


In [7]:
findAnswer("What is the currencyCode of India")
findAnswer("What is the populationCensus of England")
findAnswer("Who is Rabindranath Tagore")

result
INR
INR
result
53012500
result
http://dbpedia.org/resource/Category:Rabindranath_Tagore
